### From [TPM conventions](https://pyphi.readthedocs.io/en/latest/conventions.html)

In [1]:
import numpy as np
import pandas as pd
import pyphi
from pyphi.examples import basic_noisy_selfloop_network

import pyphi.data_models as dm  # Prototype code


Welcome to PyPhi!

If you use PyPhi in your research, please cite the paper:

  Mayner WGP, Marshall W, Albantakis L, Findlay G, Marchman R, Tononi G.
  (2018). PyPhi: A toolbox for integrated information theory.
  PLOS Computational Biology 14(7): e1006343.
  https://doi.org/10.1371/journal.pcbi.1006343

Documentation is available online (or with the built-in `help()` function):
  https://pyphi.readthedocs.io

To report issues, please use the issue tracker on the GitHub repository:
  https://github.com/wmayner/pyphi

For general discussion, you are welcome to join the pyphi-users group:
  https://groups.google.com/forum/#!forum/pyphi-users

To suppress this message, either:
  - Set `WELCOME_OFF: true` in your `pyphi_config.yml` file, or
  - Set the environment variable PYPHI_WELCOME_OFF to any value in your shell:
        export PYPHI_WELCOME_OFF='yes'



In [2]:
net = basic_noisy_selfloop_network()
tpm = net.tpm
net

Network([[[[0.271 0.19  0.244]
   [0.919 0.91  0.756]]

  [[0.919 0.91  0.756]
   [0.991 0.99  0.244]]]


 [[[0.919 0.19  0.756]
   [0.991 0.91  0.244]]

  [[0.991 0.91  0.244]
   [0.999 0.99  0.756]]]], cm=[[1 0 1]
 [1 1 1]
 [1 1 1]])

In [3]:
net.cm

array([[1, 0, 1],
       [1, 1, 1],
       [1, 1, 1]])

In [4]:
net.node_labels

NodeLabels(('n0', 'n1', 'n2'))

In [5]:
N = len(net.node_labels)
N

3

In [6]:
tpm.shape

(2, 2, 2, 3)

In [7]:
tpm

array([[[[0.271, 0.19 , 0.244],
         [0.919, 0.91 , 0.756]],

        [[0.919, 0.91 , 0.756],
         [0.991, 0.99 , 0.244]]],


       [[[0.919, 0.19 , 0.756],
         [0.991, 0.91 , 0.244]],

        [[0.991, 0.91 , 0.244],
         [0.999, 0.99 , 0.756]]]])

State-by-State form is 2^N x 2^N  (2=number of states per node)

In [8]:
sbn = tpm
sbs = pyphi.convert.sbn2sbs(tpm)
sbs.shape

(8, 8)

In [9]:
pd.DataFrame(sbs)

,0,1,2,3,4,5,6,7
0,0.446410,0.165950,0.104714,0.038926,0.144080,0.053560,0.033796,0.012564
1,0.016009,0.181631,0.003755,0.042605,0.049601,0.562759,0.011635,0.132005
2,0.001779,0.020181,0.017985,0.204055,0.005511,0.062529,0.055725,0.632235
3,0.000612,0.067428,0.006192,0.681768,0.000198,0.021762,0.001998,0.220042
4,0.001779,0.020181,0.017985,0.204055,0.005511,0.062529,0.055725,0.632235
5,0.000612,0.067428,0.006192,0.681768,0.000198,0.021762,0.001998,0.220042
6,0.000068,0.007492,0.006736,0.741704,0.000022,0.002418,0.002174,0.239386
7,0.000002,0.002438,0.000242,0.241318,0.000008,0.007552,0.000748,0.747692


In [10]:
# New version of Legacy Network
n2 = dm.Network()
n2.from_legacy(net)
n2

Network(7fef2a1027f): node_connectivity: (3, 3), transitions: (8, 8)

In [11]:
n2.tpm

,"(0, 0, 0)","(0, 0, 1)","(0, 1, 0)","(0, 1, 1)","(1, 0, 0)","(1, 0, 1)","(1, 1, 0)","(1, 1, 1)"
"(0, 0, 0)",0.446410,0.165950,0.104714,0.038926,0.144080,0.053560,0.033796,0.012564
"(0, 0, 1)",0.016009,0.181631,0.003755,0.042605,0.049601,0.562759,0.011635,0.132005
"(0, 1, 0)",0.001779,0.020181,0.017985,0.204055,0.005511,0.062529,0.055725,0.632235
"(0, 1, 1)",0.000612,0.067428,0.006192,0.681768,0.000198,0.021762,0.001998,0.220042
"(1, 0, 0)",0.001779,0.020181,0.017985,0.204055,0.005511,0.062529,0.055725,0.632235
"(1, 0, 1)",0.000612,0.067428,0.006192,0.681768,0.000198,0.021762,0.001998,0.220042
"(1, 1, 0)",0.000068,0.007492,0.006736,0.741704,0.000022,0.002418,0.002174,0.239386
"(1, 1, 1)",0.000002,0.002438,0.000242,0.241318,0.000008,0.007552,0.000748,0.747692


### The following acts roughly like [state-by-node form](https://pyphi.readthedocs.io/en/latest/conventions.html#multidimensional-state-by-node-form)
But, instead of being high dimensional, its 2 dimensions. Indexed by state given as tuple.

In [12]:
n2.tpm.loc[[(0,1,0)]]  # get one row

,"(0, 0, 0)","(0, 0, 1)","(0, 1, 0)","(0, 1, 1)","(1, 0, 0)","(1, 0, 1)","(1, 1, 0)","(1, 1, 1)"
"(0, 1, 0)",0.001779,0.020181,0.017985,0.204055,0.005511,0.062529,0.055725,0.632235


In [13]:
n2.tpm.loc[[(0,1,0)]].to_numpy()  # get one row

array([[0.00177876, 0.02018124, 0.01798524, 0.20405476, 0.00551124,
        0.06252876, 0.05572476, 0.63223524]])

In [14]:
n2.tpm.to_numpy()

array([[4.4641044e-01, 1.6594956e-01, 1.0471356e-01, 3.8926440e-02,
        1.4407956e-01, 5.3560440e-02, 3.3796440e-02, 1.2563560e-02],
       [1.6008840e-02, 1.8163116e-01, 3.7551600e-03, 4.2604840e-02,
        4.9601160e-02, 5.6275884e-01, 1.1634840e-02, 1.3200516e-01],
       [1.7787600e-03, 2.0181240e-02, 1.7985240e-02, 2.0405476e-01,
        5.5112400e-03, 6.2528760e-02, 5.5724760e-02, 6.3223524e-01],
       [6.1236000e-04, 6.7427640e-02, 6.1916400e-03, 6.8176836e-01,
        1.9764000e-04, 2.1762360e-02, 1.9983600e-03, 2.2004164e-01],
       [1.7787600e-03, 2.0181240e-02, 1.7985240e-02, 2.0405476e-01,
        5.5112400e-03, 6.2528760e-02, 5.5724760e-02, 6.3223524e-01],
       [6.1236000e-04, 6.7427640e-02, 6.1916400e-03, 6.8176836e-01,
        1.9764000e-04, 2.1762360e-02, 1.9983600e-03, 2.2004164e-01],
       [6.8040000e-05, 7.4919600e-03, 6.7359600e-03, 7.4170404e-01,
        2.1960000e-05, 2.4180400e-03, 2.1740400e-03, 2.3938596e-01],
       [2.4400000e-06, 2.4375600e-03, 2.4

In [15]:
n2.cm

,0,1,2
0,1,0,1
1,1,1,1
2,1,1,1


In [16]:
# Make TPM for 3 inputs, 1 ouput
probs = np.array([[0.68893508, 0.40176953],
                      [0.44095309, 0.84035902],
                      [0.09975545, 0.58225631],
                      [0.86475645, 0.18650795],
                      [0.50721989, 0.86299773],
                      [0.62045787, 0.90525779],
                      [0.88270204, 0.46225991],
                      [0.51548114, 0.89159624]])
probs

array([[0.68893508, 0.40176953],
       [0.44095309, 0.84035902],
       [0.09975545, 0.58225631],
       [0.86475645, 0.18650795],
       [0.50721989, 0.86299773],
       [0.62045787, 0.90525779],
       [0.88270204, 0.46225991],
       [0.51548114, 0.89159624]])

In [17]:
tp = dm.TransProb(list('ABC'),list('D'),probs)
tp.df

,"(0,)","(1,)"
"(0, 0, 0)",0.688935,0.401770
"(0, 0, 1)",0.440953,0.840359
"(0, 1, 0)",0.099755,0.582256
"(0, 1, 1)",0.864756,0.186508
"(1, 0, 0)",0.507220,0.862998
"(1, 0, 1)",0.620458,0.905258
"(1, 1, 0)",0.882702,0.462260
"(1, 1, 1)",0.515481,0.891596


In [18]:
print(f'in_nodes={tp.in_nodes}')
print(f'out_nodes={tp.out_nodes}')

in_nodes=['A', 'B', 'C']
out_nodes=['D']
